In [1]:
import pandas as pd

data = pd.read_csv("../data/design-seeds.csv")

In [2]:
multiplier = 8

In [3]:
from repalette.utils.data import PairRecolorDataset

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)
val, test = train_test_split(test, test_size=0.5)

In [17]:
from repalette.utils.data import ShuffleDataLoader

train_dataset = PairRecolorDataset(train, multiplier)
train_dataloader = ShuffleDataLoader(train_dataset, shuffle=False, num_workers=8)

In [18]:
val_dataset = PairRecolorDataset(val, multiplier)
val_dataloader = ShuffleDataLoader(val_dataset, shuffle=False, num_workers=8)

In [19]:
test_dataset = PairRecolorDataset(test, multiplier)
test_dataloader = ShuffleDataLoader(test_dataset, shuffle=False, num_workers=8)

In [20]:
BATCH_SIZE = multiplier

In [21]:
from repalette.models import PaletteNet

In [22]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import pandas as pd


from collections import OrderedDict

from repalette.model_common.blocks import ConvBlock, DeconvBlock, ResnetLayer, BasicBlock
from repalette.constants import LR, BETAS
from repalette.utils import PairRecolorDataset

In [23]:
from repalette.models import PaletteNet

In [28]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
checkpoint_callback = ModelCheckpoint(
    filepath="models",
    monitor="Val/AvgLoss",
    verbose=True,
    mode='min',
    save_top_k=2
)

early_stop_callback = EarlyStopping(
   monitor="Val/AvgLoss",
   min_delta=0.00,
   patience=5,
   verbose=False,
   mode='min'
)

hparams={
    'lr': LR, 'betas': BETAS,
    "batch_size": 32, "num_workers": 8
}
model = PaletteNet(
    train_dataloader, val_dataloader, test_dataloader,
    hparams=hparams,
)
logger = TensorBoardLogger('pl_logs', name='PaletteNet')
trainer = Trainer(
    gpus=1, # use gpu!
    logger=logger,
    callbacks=[early_stop_callback],
    checkpoint_callback=checkpoint_callback,
#     auto_lr_find="learning_rate"
) 

# trainer.tune(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [29]:
train_iter = iter(model.train_dataloader())

In [30]:
(i1, p1), (i2, p2) = next(train_iter)

In [ ]:
trainer.fit(model)


  | Name               | Type              | Params
---------------------------------------------------------
0 | feature_extractor  | FeatureExtractor  | 11 M  
1 | recoloring_decoder | RecoloringDecoder | 2 M   
2 | loss_fn            | MSELoss           | 0     
